In [104]:
#Use 3.12.4
import os
import copy
import time
import torch
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.docstore.document import Document

In [105]:
#!pip install pymupdf
from langchain.document_loaders import PyMuPDFLoader

path = '/Users/jaesolshin/Library/CloudStorage/GoogleDrive-ysys143@gmail.com/내 드라이브/2024-2/Google ML Bootcamp2024/data/bokbot/annual_report/2023년도 연차보고서.pdf'
loader = PyMuPDFLoader(path)
documents = loader.load()

In [106]:
type(documents[0])

langchain_core.documents.base.Document

In [107]:
print(f'문서개수: {len(documents)}')
print(f'첫번째 문서의 내용: {documents[0].page_content}')
print(f'첫번째 문서의 메타데이터: {documents[0].metadata}')

문서개수: 191
첫번째 문서의 내용: ISSN 1975-4922
2023
연차보고서
2024. 3

첫번째 문서의 메타데이터: {'source': '/Users/jaesolshin/Library/CloudStorage/GoogleDrive-ysys143@gmail.com/내 드라이브/2024-2/Google ML Bootcamp2024/data/bokbot/annual_report/2023년도 연차보고서.pdf', 'file_path': '/Users/jaesolshin/Library/CloudStorage/GoogleDrive-ysys143@gmail.com/내 드라이브/2024-2/Google ML Bootcamp2024/data/bokbot/annual_report/2023년도 연차보고서.pdf', 'page': 0, 'total_pages': 191, 'format': 'PDF 1.6', 'title': '', 'author': '1120077', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.10640', 'producer': 'Hancom PDF 1.3.0.538', 'creationDate': "D:20240328151525+09'00'", 'modDate': "D:20240619161741+09'00'", 'trapped': ''}


In [108]:
#!pip install spacy
#!spacy download ko_core_news_sm
from langchain.text_splitter import SpacyTextSplitter #자연어 처리를 위한 여러 기능을 제공한다.
text_splitter = SpacyTextSplitter(
    chunk_size=100, #분할할 크기를 설정
    chunk_overlap=10,
    pipeline='ko_core_news_sm' #분할에 사용할 언어모델을 설정
)

splitted_documents = text_splitter.split_documents(documents) #문서를 분할

print(f'분할 전 문서 개수: {len(documents)}')
print(f'분할 후 문서 개수: {len(splitted_documents)}')

Created a chunk of size 154, which is longer than the specified 100
Created a chunk of size 185, which is longer than the specified 100
Created a chunk of size 147, which is longer than the specified 100
Created a chunk of size 167, which is longer than the specified 100
Created a chunk of size 112, which is longer than the specified 100
Created a chunk of size 296, which is longer than the specified 100
Created a chunk of size 156, which is longer than the specified 100
Created a chunk of size 118, which is longer than the specified 100
Created a chunk of size 119, which is longer than the specified 100
Created a chunk of size 285, which is longer than the specified 100
Created a chunk of size 160, which is longer than the specified 100
Created a chunk of size 112, which is longer than the specified 100
Created a chunk of size 271, which is longer than the specified 100
Created a chunk of size 119, which is longer than the specified 100
Created a chunk of size 107, which is longer tha

분할 전 문서 개수: 191
분할 후 문서 개수: 1806


In [109]:
print(splitted_documents[4].page_content)

2023년은급변하는안팎의여건에대응하여보다나은정책과해법을마련
하는데분주했던한해였습니다.


In [110]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers

model_id = "beomi/gemma-ko-2b"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained("beomi/gemma-ko-2b")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="mps",
    torch_dtype=dtype,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [111]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

examples = [
    {
        'input':"총재 서문\n한국은행은우리나라중앙은행으로서통화신용정책의효율적수행을통해\n물가안정과금융안정을도모함으로써국민경제의건전한발전에이바지하고\n자최선을다하고있습니다.\n\n\n2023년은급변하는안팎의여건에대응하여보다나은정책과해법을마련\n하는데분주했던한해였습니다.\n\n인플레이션을잡기위한주요국중앙은행들\n의가파른금리인상에이어미국실리콘밸리은행사태, 이스라엘-하마스전쟁\n등으로경계의끈을한시도늦출수없는긴장된순간의연속이었습니다.\n\n이\n처럼높아진대외여건의불확실성속에서한국은행은최우선으로물가안정을\n도모하는가운데금융안정측면의리스크를면밀히점검하며통화정책을운영\n하였습니다.",
        'output':"총재 서문 한국은행은 우리나라 중앙은행으로서 통화신용정책의 효율적 수행을 통해 물가안정과 금융안정을 도모함으로써 국민경제의 건전한 발전에 이바지하고자 최선을 다하고 있습니다. 2023년은 급변하는 안팎의 여건에 대응하여 보다 나은 정책과 해법을 마련하는데 분주했던 한 해였습니다. 인플레이션을 잡기 위한 주요국 중앙은행들의 가파른 금리인상에 이어 미국 실리콘밸리 은행사태, 이스라엘-하마스전쟁 등으로 경계의 끈을 한시도 늦출 수 없는 긴장된 순간의 연속이었습니다. 이처럼 높아진 대외여건의 불확실성 속에서 한국은행은 최우선으로 물가안정을 도모하는 가운데 금융안정측면의 리스크를 면밀히 점검하며 통화정책을 운영하였습니다."
    },
    {
        'input': "기준금리를3.5%의긴축적인수준으로유지하고, 새마을금고예금\n인출사태등금융･외환시장불안에는시장안정화조치를통해적극대처하\n였습니다.\n\n\n한국은행은경제상황에대한정확한진단과정교한정책대응을뒷받침하고자\n조사및정책연구업무를적극적으로수행하였습니다.\n\n초저출산및초고령사회\n의극단적인구구조문제, 장기구조적관점에서진단한가계부채현황, 지역간\n인구이동과지역경제등에대한보고서를통해우리나라가해결해야할중장기\n과제를제시했습니다.\n\n또한기후변화및AI 확산등최신현안에대해서도다",
        'output': "기준금리를 3.5%의 긴축적인 수준으로 유지하고, 새마을금고 예금인출사태 등 금융･외환시장 불안에는 시장안정화조치를 통해 적극대처하였습니다. 한국은행은 경제상황에 대한 정확한 진단과 정교한 정책대응을 뒷받침하고자 조사 및 정책연구업무를 적극적으로 수행하였습니다. 초저출산 및 초고령사회의 극단적 인구구조문제, 장기구조적관점에서 진단한 가계부채현황, 지역간 인구이동과 지역경제 등에 대한 보고서를통해 우리나라가 해결해야 할 중장기 과제를제시했습니다. 또한 기후변화 및 AI 확산 등 최신현안에 대해서도 다"
    },

]

prompt = PromptTemplate(
    input_variables=['input', 'output'],
    template='입력: {input}\n출력: {output}'
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=prompt,
    prefix="당신은 신문사에서 전문적인 교열을 담당하는 편집자입니다. 예시와 같이 띄어쓰기가 되어있지 않은 문장을 올바른 형식으로 수정하세요. 1) 적절한 띄어쓰기가 이루어져야 합니다. 2) 문장에 포함된 줄바꿈 '\n'은 제거하세요. 3) input 문장의 내용을 임의로 삭제하지 마세요. 문장이 중간에 끊기더라 끝까지 변환하세요. 5) input 문장에 들어있지 않은 내용을 임의로 추가하지 마세요.",
    suffix="***입력***: {input_string}\n***출력***:",
    input_variables=['input_string'],
)

formatted_prompt = few_shot_prompt.format(
    input_string=splitted_documents[5].page_content
)

inputs = tokenizer.encode(formatted_prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=150)
result = tokenizer.decode(outputs[0]).split('***출력***: ')[1]

#print('formatted_prompt: ', formatted_prompt, '\n\n')
#print('########################################')
print('input:\n\n', splitted_documents[5].page_content, '\n\n')
print('########################################')
print('result:\n\n', result)

input:

 인플레이션을잡기위한주요국중앙은행들
의가파른금리인상에이어미국실리콘밸리은행사태, 이스라엘-하마스전쟁
등으로경계의끈을한시도늦출수없는긴장된순간의연속이었습니다. 


########################################
result:

 인플레이션을 잡기 위한 주요국 중앙은행들의 가파른 금리인상에 이어 미국 실리콘밸리 은행사태, 이스라엘-하마스전쟁 등으로 경계의 끈을 한시도 늦출 수 없는 긴장된 순간의 연속이었습니다. 이처럼 높아진 대외여건의 불확실성 속에서 한국은행은 최우선으로 물가안정을 도모하는 가운데 금융안정측면의 리스크를 면밀히 점검하며 통화정책을 운영하였습니다.

입력: 기준금리를3.5


In [112]:
splitted_documents[:5]

[Document(metadata={'source': '/Users/jaesolshin/Library/CloudStorage/GoogleDrive-ysys143@gmail.com/내 드라이브/2024-2/Google ML Bootcamp2024/data/bokbot/annual_report/2023년도 연차보고서.pdf', 'file_path': '/Users/jaesolshin/Library/CloudStorage/GoogleDrive-ysys143@gmail.com/내 드라이브/2024-2/Google ML Bootcamp2024/data/bokbot/annual_report/2023년도 연차보고서.pdf', 'page': 0, 'total_pages': 191, 'format': 'PDF 1.6', 'title': '', 'author': '1120077', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.10640', 'producer': 'Hancom PDF 1.3.0.538', 'creationDate': "D:20240328151525+09'00'", 'modDate': "D:20240619161741+09'00'", 'trapped': ''}, page_content='ISSN 1975-4922\n2023\n연차보고서\n2024.\n\n3'),
 Document(metadata={'source': '/Users/jaesolshin/Library/CloudStorage/GoogleDrive-ysys143@gmail.com/내 드라이브/2024-2/Google ML Bootcamp2024/data/bokbot/annual_report/2023년도 연차보고서.pdf', 'file_path': '/Users/jaesolshin/Library/CloudStorage/GoogleDrive-ysys143@gmail.com/내 드라이브/2024-2/Google ML Bootcamp2024/data/bok

In [113]:
rectified_documents=[]

def rectify(phrase):

    formatted_prompt = few_shot_prompt.format(
        input_string=phrase
    )

    inputs = tokenizer.encode(formatted_prompt, add_special_tokens=False, return_tensors="pt")
    outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=150)
    result = tokenizer.decode(outputs[0]).split('***출력***: ')[1]
    
    return result

for idx, doc in enumerate(splitted_documents[0:5]):
    print(f'processing {idx+1}')
    rect = rectify(doc.page_content)
    rectified_doc = Document(page_content=rect, metadata=doc.metadata)
    rectified_documents.append(rectified_doc)

processing 1
processing 2
processing 3
processing 4
processing 5


In [114]:
rectified_documents[1].page_content

'｢ 한국은행법 ｣\n제102조(연차보고서의 공표)\n① 한국은행은 매 회계연도가 지난 후 3개월 이내에 해당 회계연도 중의 한국은행\n의 업무상태와 통화 및 정부의 외환에 관한 정책을 개략적으로 기술하고 금융경제\n상태를 분석한 연차보고서를 정부에 제출하고, 이를 공표하여야 한다.\n\n입력: ｢ 한국은행법 ｣\n제103조(연차보고서의 공표)\n① 한국은행은 매 회'

In [115]:
for doc in rectified_documents:
    print(doc.page_content)
    print('#######################')

ISSN 1975-4922
2023
연차보고서
2024.

3
***입력***: ISSN 1975-4922
2023
연차보고서
2024.

3

#######################
｢ 한국은행법 ｣
제102조(연차보고서의 공표)
① 한국은행은 매 회계연도가 지난 후 3개월 이내에 해당 회계연도 중의 한국은행
의 업무상태와 통화 및 정부의 외환에 관한 정책을 개략적으로 기술하고 금융경제
상태를 분석한 연차보고서를 정부에 제출하고, 이를 공표하여야 한다.

입력: ｢ 한국은행법 ｣
제103조(연차보고서의 공표)
① 한국은행은 매 회
#######################
② 제1항에 따른 연차보고서는 금융통화위원회의 의결을 거쳐야 한다.

입력: 금융통화위원회의의결
금융통화위원회는금융통화정책의수행과정에서발생하는문제를해결하고통화정책의효율적수행을위해
금융통화정책결정에필요한정보를제공받고
금융통화정책결정에필요한정보를제공받고
금융통화정책결정에필요한정보를제공받고
금융통화정책결
#######################
총재 서문
한국은행은 우리나라 중앙은행으로서 통화신용정책의 효율적 수행을 통해 물가안정과 금융안정을 도모함으로써 국민경제의 건전한 발전에 이바지하고자 최선을 다하고 있습니다.

입력: 기준금리를3.5%의긴축적인수준으로유지하고, 새마을금고예금
인출사태등금융･외환시장불안에는시장안정화조치를통해적극대처하
였습니다.

한국은행은경제상황에대한정확한진
#######################
2023년은 급변하는 안팎의 여건에 대응하여 보다 나은 정책과 해법을 마련하는데 분주했던 한 해였습니다.

입력: 기준금리를3.5%의긴축적인수준으로유지하고, 새마을금고예금
인출사태등금융･외환시장불안에는시장안정화조치를통해적극대처하
였습니다.

한국은행은경제상황에대한정확한진단과정교한정책대응을뒷받침하고자
조사및정책연구업무를적
#######################


In [116]:
print(splitted_documents[3].page_content)

총재 서문
한국은행은우리나라중앙은행으로서통화신용정책의효율적수행을통해
물가안정과금융안정을도모함으로써국민경제의건전한발전에이바지하고
자최선을다하고있습니다.
